# kNN Hash Example

In [23]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from functools import partial
from random import random
from sklearn.preprocessing import MinMaxScaler
from collections import defaultdict
from collections import Counter
import math

## Iris dataset

In [24]:
df = load_iris()
df.data.shape

(150, 4)

In [25]:
def f_hash(w,r,b,x):
    return int((np.dot(w,x)+b)/r)

def eucliddist(x, y): # euclid
    retVal = 0
    for i in range(x.shape[0]):
        retVal += pow(x[i] - y[0][i], 2)
    return math.sqrt(retVal)

def manhattandist(x, y): # manhattan
    retVal = 0
    for i in range(x.shape[0]):
        retVal += (x[i] - y[0][i])
    return retVal

def minkowskidist(x, y, p): # minkowski
    retVal = 0
    for i in range(x.shape[0]):
        retVal += pow(x[i] - y[0][i], p)
    return pow(retVal, 1/p)

* https://docs.python.org/2/library/functools.html Here you can read about "partial"
* http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html About mapping to [0,1]

In [26]:
class KNNHash(object):
    def __init__(self,m,L,nn):
        self.m = m
        self.L = L
        self.nn = nn

    def fit(self,X,y):
        self.t_hh = [] #hash table
        for j in range(self.L):
            f_hh = [] #compositional hash function
            for i in range(self.m):
                w = np.random.rand(1,X[0].shape[0]) #  weights of a hash function
                f_hh.append(partial(f_hash,w = w,r=random(),b=random())) # list of initialized hash function
            self.t_hh.append(
                (defaultdict(list),f_hh)
            )
        for n in range(X.shape[0]): 
            for j in range(self.L):
                ind = 0
                for i in range(self.m):
                    ind = ind + self.t_hh[j][1][i](x=X[n]) #calculation of index in hash table, simply sum of all hash func
                self.t_hh[j][0][ind].append((X[n],y[n])) #saving sample into corresponding index
    
    def predict1(self,u):
        for j in range(self.L):
            inds = []
            holder = []
            labelHolder = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            print(cntr)
            for out in self.t_hh[j][0][sum(inds)]:
                holder.append(eucliddist(u, out)) 
                labelHolder.append(out[1]) 
            answer1 = holder.index(min(holder)) 
            print(min(holder))
            print("Output: " + str(labelHolder[answer1])) 
            
    def predict2(self,u): 
        for j in range(self.L):
            inds = []
            holder = []
            labelHolder = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
            print(cntr)
            for out in self.t_hh[j][0][sum(inds)]: 
                holder.append(manhattandist(u, out)) 
                labelHolder.append(out[1])
            answer1 = holder.index(min(holder)) 
            print(min(holder))
            print("Output: " + str(labelHolder[answer1]))
            
    def predict3(self,u,p):    
        for j in range(self.L):
            inds = []
            holder = []
            labelHolder = []
            for i in range(self.m):
                inds.append(self.t_hh[j][1][i](x=u))
            cntr = Counter([outp for inpt,outp in self.t_hh[j][0][sum(inds)]])
           
            for out in self.t_hh[j][0][sum(inds)]: 
                holder.append(minkowskidist(u, out ,p))
                labelHolder.append(out[1]) 
            answer1 = holder.index(min(holder)) 
            print(min(holder)) 
            print("Output: " + str(labelHolder[answer1])) 

In [34]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target

knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict1(test1x)
print("-------------")
print(test2y)
knnhash.predict1(test2x)
print("-------------")
print(test3y)
knnhash.predict1(test3x)

0
Counter({0: 4})
0.03254041656427031
Output: 0
Counter({0: 8})
0.04498205067597233
Output: 0
Counter({0: 18, 1: 2})
0.03254041656427031
Output: 0
Counter({0: 14, 1: 2})
0.03254041656427031
Output: 0
-------------
1
Counter({1: 2, 2: 1})
0.06798027147519138
Output: 1
Counter({1: 3})
0.06798027147519138
Output: 1
Counter({1: 13, 2: 12})
0.05007710104811091
Output: 1
Counter({2: 5, 1: 4})
0.05007710104811091
Output: 1
-------------
2
Counter({2: 3, 1: 2})
0.06508083312853999
Output: 2
Counter({1: 2, 2: 2})
0.06508083312853999
Output: 2
Counter({1: 13, 2: 12})
0.05794021820301701
Output: 2
Counter({1: 8, 2: 8, 0: 1})
0.05794021820301701
Output: 2


* Each string above corresponds to the particular hash table. And index in counter maps to the class. For example Counter({0: 13, 1: 1}) means that there are 13 samples close to "u" with "0" class labels and 1 sample with "1" class label.

In [35]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target

knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict2(test1x)
print("-------------")
print(test2y)
knnhash.predict2(test2x)
print("-------------")
print(test3y)
knnhash.predict2(test3x)

0
Counter({0: 19, 1: 1})
-0.3759416195856874
Output: 1
Counter({0: 40})
-0.4027777777777779
Output: 0
Counter({0: 11})
-0.04472693032015079
Output: 0
Counter({0: 16})
-0.18361581920903963
Output: 0
-------------
1
Counter({2: 9, 1: 7})
-0.2450564971751411
Output: 2
Counter({1: 14, 2: 1})
0.0494350282485877
Output: 1
Counter({1: 6, 2: 4})
-0.12335216572504715
Output: 1
Counter({2: 10, 1: 2})
-0.06779661016949157
Output: 2
-------------
2
Counter({2: 9, 1: 7})
-0.1541902071563086
Output: 2
Counter({1: 6, 2: 6})
-0.16054613935969853
Output: 1
Counter({1: 5, 2: 5})
0.07250470809792875
Output: 2
Counter({2: 10, 1: 2})
0.02306967984934094
Output: 2


In [40]:
scaler = MinMaxScaler()
scaler.fit(df.data)
x = scaler.transform(df.data)
y = df.target

knnhash = KNNHash(4,4,4)
test1x = x[0]
test2x = x[75]
test3x = x[149]

test1y = y[0]
test2y = y[75]
test3y = y[149]
x = np.delete(x,[0,75,149],axis=0)
y = np.delete(y,[0,75,149],axis=0)

knnhash.fit(x,y)
print(test1y)
knnhash.predict3(test1x,4)
print("-------------")
print(test2y)
knnhash.predict3(test2x,4)
print("-------------")
print(test3y)
knnhash.predict3(test3x,4)

0
0.028694025020721027
Output: 0
0.04166666666666666
Output: 0
0.028694025020721027
Output: 0
0.043587347121041654
Output: 0
-------------
1
0.05206560298458324
Output: 1
0.043587347121041516
Output: 1
0.05206560298458324
Output: 1
0.21828706579220775
Output: 1
-------------
2
0.05194369203790213
Output: 2
0.05194369203790213
Output: 2
0.11994915165688995
Output: 2
0.10077475988795855
Output: 2
